# Clean Cluster 14

### Imports

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

### Define Paths

In [2]:
CWD = Path.cwd()
ROOT = CWD.parent if CWD.name.lower() == "notebooks" else CWD

DATA = ROOT / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"

RAW.mkdir(parents=True, exist_ok=True)
PROCESSED.mkdir(parents=True, exist_ok=True)

print("Python:", sys.version.split()[0])
print("RAW:", RAW)
print("PROCESSED:", PROCESSED)

Python: 3.11.9
RAW: C:\Users\danci\Interconnection-Queue-Intelligence\data\raw
PROCESSED: C:\Users\danci\Interconnection-Queue-Intelligence\data\processed


### Select Cluster 14 File

In [4]:
raw_files = sorted([p for p in RAW.iterdir() if p.is_file() and not p.name.startswith("~$")])

candidates = [p for p in raw_files if ("cluster" in p.name.lower() and "14" in p.name.lower())]
cluster14_path = candidates[0] if candidates else None

print("Detected:", cluster14_path.name if cluster14_path else None)
if cluster14_path is None:
    raise FileNotFoundError("Could not find Cluster 14 file. Make sure filename includes 'cluster' and '14'.")

Detected: PreliminaryCluster14ProjectListasofMay20-2021.xlsx


### Helpers

In [5]:
def coerce_float(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace(",", "")
    if s == "": return np.nan
    return pd.to_numeric(s, errors="coerce")

def clean_text(x):
    if pd.isna(x): return ""
    return str(x).strip()

def normalize_tech(x):
    s = clean_text(x).lower()
    if s == "": return ""
    if any(k in s for k in ["battery", "storage", "bess"]): return "storage"
    if any(k in s for k in ["solar", "pv", "photovoltaic"]): return "solar"
    if "wind" in s: return "wind"
    return clean_text(x)

### Load Sheets

In [6]:
xls = pd.ExcelFile(cluster14_path)
display(Markdown(f"**Workbook:** `{cluster14_path.name}`"))
display(pd.DataFrame({"sheet_in_file": xls.sheet_names}))

# Tailored: ignore Notes, use the overview sheet
target_sheet = "C14 Prelim Cluster Overview"
if target_sheet not in xls.sheet_names:
    raise ValueError(f"Expected sheet not found: {target_sheet}")

df14_raw = pd.read_excel(cluster14_path, sheet_name=target_sheet)

display(Markdown(f"**Raw shape:** {df14_raw.shape}"))
display(df14_raw.head(5))
display(pd.DataFrame({"column": df14_raw.columns.astype(str).tolist()}))

**Workbook:** `PreliminaryCluster14ProjectListasofMay20-2021.xlsx`

,sheet_in_file
0,Notes
1,C14 Prelim Cluster Overview


**Raw shape:** (363, 12)

,PTO,Affected PTO,Area,Total MW @ POI,Technology\n#1,MW-T1,Technology\n#2,MW-T2,Technology\n#3,MW-T3,POI,Voltage\n(kV)
0,DCRT,SCE,Eastern,500.0,Battery,516.7638,Solar PV,516.7638,NaN,NaN,Cielo Azul Substation,500.0
1,DCRT,SCE,Eastern,350.0,Battery,350.0000,Solar PV,350.0000,NaN,NaN,Delaney Substation,500.0
2,DCRT,SCE,Eastern,2000.0,Battery,2000.0000,NaN,NaN,NaN,NaN,Cielo Azul Substation,500.0
3,DCRT,SCE,Eastern,350.0,Solar PV,357.5300,NaN,NaN,NaN,NaN,Delaney-Colorado River,500.0
4,DCRT,SCE,Eastern,700.0,Battery,718.8100,Solar PV,718.8100,NaN,NaN,Delaney-Colorado River,500.0


,column
0,PTO
1,Affected PTO
2,Area
3,Total MW @ POI
4,Technology\n#1
5,MW-T1
6,Technology\n#2
7,MW-T2
8,Technology\n#3
9,MW-T3


### Rename + Keep Columns

In [11]:
# Your provided Cluster 14 columns:
# PTO, Affected PTO, Area, Total MW @ POI,
# Technology #1, MW-T1, Technology #2, MW-T2, Technology #3, MW-T3,
# POI, Voltage (kV)

RENAME_14 = {
    "PTO": "pto",
    "Affected PTO": "affected_pto",   # optional, not in public/cluster15, but keep (could be useful later)
    "Area": "study_area",
    "Total MW @ POI": "mw_poi",
    "Technology\n#1": "tech_1",
    "MW-T1": "mw_1",
    "Technology\n#2": "tech_2",
    "MW-T2": "mw_2",
    "Technology\n#3": "tech_3",
    "MW-T3": "mw_3",
    "POI": "poi",
    "Voltage\n(kV": "voltage_kv",      # note: sometimes the column is missing the closing parenthesis
    "Voltage\n(kV)": "voltage_kv",     # include this just in case the file has it
}

present = {k: v for k, v in RENAME_14.items() if k in df14_raw.columns}
missing = [k for k in RENAME_14.keys() if k not in df14_raw.columns]

df14 = df14_raw.rename(columns=present).copy()

display(Markdown("**Missing expected columns (if any):**"))
display(pd.DataFrame({"missing": missing}) if missing else pd.DataFrame({"missing": ["(none)"]}))

**Missing expected columns (if any):**

,missing
0,Voltage\n(kV


### Standardize to Canonical

In [8]:
# Identity
df14["source"] = "cluster14"
df14["sheet_outcome"] = "intake_snapshot"
df14["status_raw"] = ""  # for schema compatibility later

# Clean strings
for c in ["pto", "affected_pto", "study_area", "poi", "status_raw"]:
    if c in df14.columns:
        df14[c] = df14[c].apply(clean_text)

# Normalize tech
for t in ["tech_1", "tech_2", "tech_3"]:
    if t in df14.columns:
        df14[t] = df14[t].apply(normalize_tech)

# Coerce numeric
for m in ["mw_poi", "mw_1", "mw_2", "mw_3", "voltage_kv"]:
    if m in df14.columns:
        df14[m] = df14[m].apply(coerce_float)

# Canonical-ish column order
CANON_COLS = [
    "source", "sheet_outcome",
    "pto", "affected_pto", "study_area", "poi", "voltage_kv",
    "tech_1", "mw_1", "tech_2", "mw_2", "tech_3", "mw_3",
    "mw_poi",
    "status_raw",
]
CANON_COLS = [c for c in CANON_COLS if c in df14.columns]
df14 = df14[CANON_COLS].copy()

display(Markdown(f"**Standardized shape:** {df14.shape}"))
display(df14.head(5))

**Standardized shape:** (363, 15)

,source,sheet_outcome,pto,affected_pto,study_area,poi,voltage_kv,tech_1,mw_1,tech_2,mw_2,tech_3,mw_3,mw_poi,status_raw
0,cluster14,intake_snapshot,DCRT,SCE,Eastern,Cielo Azul Substation,500.0,storage,516.7638,solar,516.7638,,NaN,500.0,
1,cluster14,intake_snapshot,DCRT,SCE,Eastern,Delaney Substation,500.0,storage,350.0000,solar,350.0000,,NaN,350.0,
2,cluster14,intake_snapshot,DCRT,SCE,Eastern,Cielo Azul Substation,500.0,storage,2000.0000,,NaN,,NaN,2000.0,
3,cluster14,intake_snapshot,DCRT,SCE,Eastern,Delaney-Colorado River,500.0,solar,357.5300,,NaN,,NaN,350.0,
4,cluster14,intake_snapshot,DCRT,SCE,Eastern,Delaney-Colorado River,500.0,storage,718.8100,solar,718.8100,,NaN,700.0,


### QA Checks

In [9]:
# Basic missingness and distribution checks
fields = ["mw_poi", "voltage_kv", "pto", "study_area", "poi", "tech_1"]
qa = []
for f in fields:
    if f in df14.columns:
        qa.append((f, df14[f].isna().mean(), df14[f].nunique(dropna=True)))
display(pd.DataFrame(qa, columns=["field", "share_missing", "n_unique"]))

if "mw_poi" in df14.columns:
    display(Markdown("**MW POI summary:**"))
    display(df14["mw_poi"].describe().to_frame().T)

if "tech_1" in df14.columns:
    display(Markdown("**Tech slot 1 counts:**"))
    display(df14["tech_1"].value_counts(dropna=False).to_frame("count"))

,field,share_missing,n_unique
0,mw_poi,0.000000,78
1,voltage_kv,0.002755,10
2,pto,0.000000,6
3,study_area,0.000000,11
4,poi,0.000000,237
5,tech_1,0.000000,5


**MW POI summary:**

,count,mean,std,min,25%,50%,75%,max
mw_poi,363.0,291.998033,263.516511,0.0,125.0,210.0,400.0,2000.0


**Tech slot 1 counts:**

,count
tech_1,
storage,336
solar,15
wind,9
Steam Turbine,2
Steam Turbine Geothermal,1


### Save Output

In [12]:
out_path = PROCESSED / "cluster14_clean.csv"
df14.to_csv(out_path, index=False)

display(Markdown(f"**Saved:** `{out_path}`"))
display(df14.head(50))

**Saved:** `C:\Users\danci\Interconnection-Queue-Intelligence\data\processed\cluster14_clean.csv`

,pto,affected_pto,study_area,mw_poi,tech_1,mw_1,tech_2,mw_2,tech_3,mw_3,poi,voltage_kv
0,DCRT,SCE,Eastern,500.0,Battery,516.7638,Solar PV,516.7638,NaN,NaN,Cielo Azul Substation,500.0
1,DCRT,SCE,Eastern,350.0,Battery,350.0000,Solar PV,350.0000,NaN,NaN,Delaney Substation,500.0
2,DCRT,SCE,Eastern,2000.0,Battery,2000.0000,NaN,NaN,NaN,NaN,Cielo Azul Substation,500.0
3,DCRT,SCE,Eastern,350.0,Solar PV,357.5300,NaN,NaN,NaN,NaN,Delaney-Colorado River,500.0
4,DCRT,SCE,Eastern,700.0,Battery,718.8100,Solar PV,718.8100,NaN,NaN,Delaney-Colorado River,500.0
5,DCRT,SCE,Eastern,500.0,Battery,508.5600,Solar PV,508.5600,NaN,NaN,Cielo Azul Substation,500.0
6,DCRT,SCE,Eastern,400.0,Battery,410.7600,Solar PV,410.7600,NaN,NaN,Delaney Substation,500.0
7,DCRT,SCE,Eastern,450.0,Battery,450.0000,Solar PV,450.0000,NaN,NaN,Delaney Substation,500.0
8,DCRT,SCE,Eastern,150.0,Battery,150.0000,Solar PV,175.0000,NaN,NaN,Delaney,500.0
9,SCE,NaN,Eastern,250.0,Battery,220.0000,NaN,NaN,NaN,NaN,Vista Substation,230.0
